In [1]:
import numpy as np
import zarr
import pandas as pd
import dask.array as da
import allel
import matplotlib.pyplot as plt
%matplotlib inline

/opt/conda/lib/python3.6/site-packages/dask/config.py:161: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
storage_path = 'ag1000g-release/phase2.AR1/variation/main/zarr/all/ag1000g.phase2.ar1'

In [3]:
import gcsfs
gcs_bucket_fs = gcsfs.GCSFileSystem(project='malariagen-jupyterhub', token='anon', access='read_only')
store = gcsfs.mapping.GCSMap(storage_path, gcs=gcs_bucket_fs, check=False, create=False)


In [4]:
callset = zarr.open_consolidated(store)
callset

<zarr.hierarchy.Group '/'>

In [41]:
variants = callset['3L/variants']

In [42]:
variants

<zarr.hierarchy.Group '/3L/variants'>

In [43]:
variants['POS']

<zarr.core.Array '/3L/variants/POS' (18167056,) int32>

In [44]:
variants['DP']

<zarr.core.Array '/3L/variants/DP' (18167056,) int32>

In [45]:
variants['AC']

<zarr.core.Array '/3L/variants/AC' (18167056, 3) int32>

In [26]:
import dask.array as da

abhet = da.from_zarr(callset['3L']['variants']['ABHet'])#
dp = da.from_zarr(callset['3L']['variants']['DP'])

pos = da.from_zarr(callset['3L']['variants']['POS'])



In [36]:
abhet

dask.array<zarr, shape=(18167056,), dtype=float32, chunksize=(4194304,)>

In [38]:
x = da.ones((1000, 3), chunks=(100, 3))

x_1 = x[:, 0].to_dask_dataframe(columns="x_1")
x_1

Dask Series Structure:
npartitions=10
0      float64
100        ...
        ...   
900        ...
999        ...
Name: x_1, dtype: float64
Dask Name: from-dask, 30 tasks

In [39]:
x.to_dask_dataframe(columns=["x_1", "x_2", "x_3"])

,x_1,x_2,x_3
npartitions=10,,,
0,float64,float64,float64
100,...,...,...
...,...,...,...
900,...,...,...
999,...,...,...


In [40]:
pd.DataFrame({
    'x': np.ones((100, 3)),
})

ValueError: If using all scalar values, you must pass an index

In [ ]:
da.from_array
da.Array.to_dask_dataframe
dd.concat

In [35]:
abhet.to_dask_dataframe()

Dask Series Structure:
npartitions=5
0           float32
4194304         ...
             ...   
16777216        ...
18167055        ...
dtype: float32
Dask Name: from-dask, 11 tasks

In [29]:
import dask.dataframe as dd

df = dd.concat([
    abhet.to_dask_dataframe(columns="ABHet"),
    dp.to_dask_dataframe(columns="DP"),
    pos.to_dask_dataframe(columns="POS"),
], axis=1).set_index("POS", sorted=True)
df

,ABHet,DP
npartitions=5,,
15,float32,int32
15718176,...,...
...,...,...
39194113,...,...
41963345,...,...


In [34]:
df.loc[29000000:300000000]

,ABHet,DP
npartitions=3,,
29000000,float32,int32
31388966,...,...
39194113,...,...
41963345,...,...


In [30]:
df.divisions

(15, 15718176, 24027935, 31388966, 39194113, 41963345)

In [32]:
df.describe().compute()

,ABHet,DP
count,1.291516e+07,1.816706e+07
mean,5.551281e-01,3.227796e+04
std,1.179507e-01,5.048468e+03
min,3.600000e-02,1.000000e+00
25%,4.900000e-01,3.377800e+04
50%,5.340000e-01,3.719700e+04
75%,6.360000e-01,4.008500e+04
max,9.960000e-01,2.837780e+05


In [33]:
df[df.DP > 50000].compute()

,ABHet,DP
POS,,
697,NaN,55212
711,NaN,58212
721,0.783,62640
723,0.789,63550
732,0.806,65734
739,0.825,68876
744,0.891,69719
748,0.896,69192
756,0.749,65076


In [13]:
abhet.to_dask_dataframe?

Signature: abhet.to_dask_dataframe(columns=None, index=None)
Docstring:
Convert dask Array to dask Dataframe

Parameters
----------
columns: list or string
    list of column names if DataFrame, single string if Series
index : dask.dataframe.Index, optional
    An optional *dask* Index to use for the output Series or DataFrame.

    The default output index depends on whether the array has any unknown
    chunks. If there are any unknown chunks, the output has ``None``
    for all the divisions (one per chunk). If all the chunks are known,
    a default index with known divsions is created.

    Specifying ``index`` can be useful if you're conforming a Dask Array
    to an existing dask Series or DataFrame, and you would like the
    indices to match.

See Also
--------
dask.dataframe.from_dask_array
File:      /opt/conda/lib/python3.6/site-packages/dask/array/core.py
Type:      method


In [11]:
x.to_dask_dataframe()

Dask Series Structure:
npartitions=5
0           float32
4194304         ...
             ...   
16777216        ...
18167055        ...
dtype: float32
Dask Name: from-dask, 11 tasks

In [ ]:
import dask.array as da

x = 

## Cluster setup

In [2]:
from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=30)
cluster

/opt/conda/lib/python3.6/site-packages/dask_kubernetes/config.py:13: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)
/opt/conda/lib/python3.6/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)
/opt/conda/lib/python3.6/site-packages/bokeh/themes/theme.py:131: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  json = yaml.load(f)


In [3]:
from dask.distributed import Client
client = Client(cluster)
client

Client Scheduler: tcp://10.32.1.80:45493 Dashboard: /user/alimanfoo@googlemail.com/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


## Data setup

In [17]:
storage_path = 'ag1000g-release/phase2.AR1/variation/main/zarr/all/ag1000g.phase2.ar1'

# GCS configuration
import gcsfs
gcs_bucket_fs = gcsfs.GCSFileSystem(project='malariagen-jupyterhub', token='anon', access='read_only')
store = gcsfs.mapping.GCSMap(storage_path, gcs=gcs_bucket_fs, check=False, create=False)

# S3 or compatible object storage configuration
#import s3fs
#s3 = s3fs.S3FileSystem(anon=True, client_kwargs=dict(region_name='eu-west-2'))
#store = s3fs.S3Map(root=storage_path, s3=s3, check=False)

In [18]:
callset = zarr.open_consolidated(store)
callset

<zarr.hierarchy.Group '/'>

In [19]:
chrom = '3R'

In [20]:
gtz = callset[chrom]['calldata/GT']
gtz

<zarr.core.Array '/3R/calldata/GT' (24943504, 1142, 2) int8>

In [21]:
# gtz.info

In [22]:
gt = allel.GenotypeDaskArray(gtz)
gt

<GenotypeDaskArray shape=(24943504, 1142, 2) dtype=int8>

In [23]:
!wget --no-clobber ftp://ngs.sanger.ac.uk/production/ag1000g/phase2/AR1/samples/samples.meta.txt

File ‘samples.meta.txt’ already there; not retrieving.


In [24]:
df_samples = pd.read_csv('samples.meta.txt', sep='\t')
df_samples.head()

,ox_code,src_code,population,country,location,site,contributor,contact,year,m_s,sex,n_sequences,mean_coverage,ebi_sample_acc,latitude,longitude
0,AA0040-C,Twifo_Praso__E2,GHcol,Ghana,Twifo Praso,Twifo Praso,David Weetman,David Weetman,2012,M,F,95033368,30.99,ERS311878,5.60858,-1.54926
1,AA0041-C,Twifo_Praso__H3,GHcol,Ghana,Twifo Praso,Twifo Praso,David Weetman,David Weetman,2012,M,F,95843804,31.70,ERS311886,5.60858,-1.54926
2,AA0042-C,Takoradi_C7,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,107420666,35.65,ERS311894,4.91217,-1.77397
3,AA0043-C,Takoradi_H8,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,95993752,29.46,ERS311902,4.91217,-1.77397
4,AA0044-C,Takoradi_D10,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,103044262,33.67,ERS311910,4.91217,-1.77397


## Subset data

In [25]:
pop = 'GHcol'

In [26]:
loc_pass_variants = callset[chrom]['variants/FILTER_PASS'][:]
loc_pass_variants

array([False, False, False, ..., False, False, False])

In [27]:
len(loc_pass_variants), np.count_nonzero(loc_pass_variants)

(24943504, 14481509)

In [28]:
loc_pop_samples = df_samples[df_samples.population == pop].index.values
loc_pop_samples

array([ 0,  1,  2,  3,  4,  6,  8,  9, 10, 11, 12, 13, 16, 17, 18, 19, 20,
       22, 23, 24, 25, 26, 27, 31, 32, 33, 34, 35, 37, 38, 39, 40, 41, 42,
       43, 44, 45, 47, 48, 49, 50, 51, 52, 53, 55, 56, 57, 58, 59, 61, 62,
       63, 64, 65, 66])

In [29]:
len(loc_pop_samples)

55

In [30]:
gt_pass_pop = gt.subset(loc_pass_variants, loc_pop_samples)
gt_pass_pop

<GenotypeDaskArray shape=(14481509, 55, 2) dtype=int8>

## Allele count computation

In [31]:
# watch the dask dashboard while this is computing
ac_pass_pop = gt_pass_pop.count_alleles(max_allele=3).compute()
ac_pass_pop

<AlleleCountsArray shape=(14481509, 4) dtype=int32>
110   0   0   0
110   0   0   0
110   0   0   0
...
110   0   0   0
110   0   0   0
110   0   0   0

In [32]:
ac_pass_pop.count_segregating()

3740771

## Multi-population test for selection

In [35]:
def population_allele_counts(chrom, pop):
    gtz = callset[chrom]['calldata/GT']
    gt = allel.GenotypeDaskArray(gtz)
    loc_pass_variants = callset[chrom]['variants/FILTER_PASS'][:]
    loc_pop_samples = df_samples[df_samples.population == pop].index.values
    gt_pass_pop = gt.subset(loc_pass_variants, loc_pop_samples)
    ac_pass_pop = gt_pass_pop.count_alleles(max_allele=3)
    return ac_pass_pop


def pbs(chrom, pop1, pop2, pop3, window_size=100, min_maf=0.02, normed=True):
    
    # load variant positions
    loc_pass_variants = callset[chrom]['variants/FILTER_PASS'][:]
    pos = callset[chrom]['variants/POS'][:][loc_pass_variants]
    
    # load allele counts
    ac1 = population_allele_counts(chrom, pop1)
    ac2 = population_allele_counts(chrom, pop2)
    ac3 = population_allele_counts(chrom, pop3)
    ac1, ac2, ac3 = da.compute(ac1, ac2, ac3)
    ac1 = allel.AlleleCountsArray(ac1)
    ac2 = allel.AlleleCountsArray(ac2)
    ac3 = allel.AlleleCountsArray(ac3)

    # locate segregating variants at sufficient frequency
    ac = ac1 + ac2 + ac3
    loc_seg = ac.is_biallelic_01() & (ac.to_frequencies()[:, :2].min(axis=1) > min_maf)
    pos = pos[loc_seg]
    ac1 = ac1[loc_seg]
    ac2 = ac2[loc_seg]
    ac3 = ac3[loc_seg]
    
    # setup windows
    starts = allel.moving_statistic(pos, statistic=lambda v: v[0], size=window_size)
    starts[0] = 1  # fix to start of sequence
    ends = np.append(starts[1:] - 1, [np.max(pos)])

    # compute pbs
    res = allel.pbs(ac1, ac2, ac3, window_size=window_size, normed=normed)

    return starts, ends, res


In [ ]:
# watch the dask dashboard
starts, ends, y = pbs('3R', 'BFgam', 'UGgam', 'GW')

In [ ]:
fig, ax = plt.subplots(figsize=(14, 3))
x = (starts + ends) / 2
ax.plot(x, y, marker='o', linestyle=' ', mfc='none', mec='k', markersize=2)
ax.set_xlabel('Genome position (bp)')
ax.set_ylabel('PBS');